#  Character-Based Neural Language Model in Keras

 A language model predicts the next word in the sequence based on the specific words that have come before it in the sequence. This comes at the cost of requiring larger models that are slower to train. Nevertheless, in the field of neural language models, character-based models offer a lot of promise for a general, flexible and powerful approach to language modeling. In this section you will know:
 How to prepare text for character-based language modeling.
 How to develop a character-based language model using LSTMs.
 How to use a trained character-based language model to generate text


# Data Preparation

The first step is to prepare the text data. We will start by defining the type of language model.

In [1]:
from numpy import array
import tensorflow as tf
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
# define the model

/home/raul/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Clean Text 
Next, we need to clean the loaded text. We will not do much to it on this example. Specifically,
we will strip all of the new line characters so that we have one long sequence of characters
separated only by white space.



In [3]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

# load text
raw_text = load_doc('elogio-ruben.txt')
#raw_text = load_doc('/floyd/input/dataset2/el_quijote.txt')
#raw_text = load_doc('rhyme.txt')

#print(raw_text)
# clean
tokens = raw_text.split()
raw_text = ' '.join(tokens)


# Create Sequences
Now that we have a long list of characters, we can create our input-output sequences used to
train the model. Each input sequence will be 10 characters with one output character, making
each sequence 11 characters long. We can create the sequences by enumerating the characters
in the text, starting at the 11th character at index 10. The sequences are save in a file with
function save_doc()

In [4]:
# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):
	# select sequence of tokens
	seq = raw_text[i-length:i+1]
	# store
	sequences.append(seq)
print('Total Sequences: %d' % len(sequences))
# save sequences to file
out_filename = 'char_sequences.txt'
print (sequences)
save_doc(sequences, out_filename)

Total Sequences: 1886
['Elogio de l', 'logio de la', 'ogio de la ', 'gio de la s', 'io de la se', 'o de la seg', ' de la segu', 'de la segui', 'e la seguid', ' la seguidi', 'la seguidil', 'a seguidill', ' seguidilla', 'seguidilla ', 'eguidilla M', 'guidilla Me', 'uidilla Met', 'idilla Metr', 'dilla Metro', 'illa Metro ', 'lla Metro m', 'la Metro má', 'a Metro mág', ' Metro mági', 'Metro mágic', 'etro mágico', 'tro mágico ', 'ro mágico y', 'o mágico y ', ' mágico y r', 'mágico y ri', 'ágico y ric', 'gico y rico', 'ico y rico ', 'co y rico q', 'o y rico qu', ' y rico que', 'y rico que ', ' rico que a', 'rico que al', 'ico que al ', 'co que al a', 'o que al al', ' que al alm', 'que al alma', 'ue al alma ', 'e al alma e', ' al alma ex', 'al alma exp', 'l alma expr', ' alma expre', 'alma expres', 'lma expresa', 'ma expresas', 'a expresas ', ' expresas l', 'expresas ll', 'xpresas lla', 'presas llam', 'resas llame', 'esas llamea', 'sas llamean', 'as llameant', 's llameante', ' llameantes', 'l

# Train Language Model

The model will read encoded characters and predict the next character in the sequence. The first step is to load the prepared character sequence data from char sequences.txt. 

In [5]:

# load doc into memory
def load_doc(filename):
  # open the file as read only
  file = open(filename, 'r' )
  # read all text
  text = file.read()
  # close the file
  file.close()
  return text
# load
in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split( '\n' )
#print lines


# Dictionary Mapping
We can create the mapping given a sorted set of unique characters in the
raw input data. The mapping is a dictionary of character values to integer values.
 

In [6]:
# integer encode sequences of characters
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))
sequences = list()
for line in lines:
	# integer encode line
	encoded_seq = [mapping[char] for char in line]
	# store
	sequences.append(encoded_seq)
# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)
print(mapping)

Vocabulary Size: 50
{'\n': 0, ' ': 1, ',': 2, '-': 3, '.': 4, ':': 5, ';': 6, 'A': 7, 'C': 8, 'D': 9, 'E': 10, 'L': 11, 'M': 12, 'O': 13, 'P': 14, 'Q': 15, 'R': 16, 'S': 17, 'T': 18, 'V': 19, 'a': 20, 'b': 21, 'c': 22, 'd': 23, 'e': 24, 'f': 25, 'g': 26, 'h': 27, 'i': 28, 'j': 29, 'l': 30, 'm': 31, 'n': 32, 'o': 33, 'p': 34, 'q': 35, 'r': 36, 's': 37, 't': 38, 'u': 39, 'v': 40, 'x': 41, 'y': 42, 'z': 43, 'á': 44, 'é': 45, 'í': 46, 'ñ': 47, 'ó': 48, 'ú': 49}


# Encode Sequences
The sequences of characters must be encoded as integers. This means that each unique character
will be assigned a specific integer value and each sequence of characters will be encoded as a
sequence of integers. We can separate the columns into input and
output sequences of characters. We can do this using a simple array slice.

In [7]:
# separate into input and output
sequences = array(sequences)
print('vectorización de secuencias')
#print (sequences[:,:-1],sequences[:,-1])
X, y = sequences[:,:-1], sequences[:,-1]
#print ('estas son las secuenciassssss', X,'de yyyyyy',y)
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)
print (X[0],X[1],'este es el',y[0],y[1])
# define model


vectorización de secuencias
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0

# Define the language model
The model is defined with an input layer that takes sequences that have 10 time steps and 38
features for the one hot encoded input sequences. The model has a single LSTM hidden layer with 75 memory cells, chosen with a little trial and error. The model has a fully connected output layer that outputs one vector with a probability distribution across all characters in the vocabulary. A softmax activation function is used on
the output layer to ensure the output has the properties of a probability distribution.


In [9]:
from pickle import dump
import tensorflow as tf
def define_model(X):
    model = Sequential()
    model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(vocab_size, activation= 'softmax' ))
    # compile model
    model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
    # summarize defined model
    model.summary()
    plot_model(model, to_file= 'model.png' , show_shapes=True)
    return model
model = define_model(X)
# fit model
model.fit(X, y, epochs=100, verbose=2)
# save the model to file
model.save( '/media/raul/datos/clases-pln/modelo-caracter/model.h5' )
# save the mapping
dump(mapping, open( '/media/raul/datos/clases-pln/modelo-caracter/mapping.pkl' , 'wb' ))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 75)                37800     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3800      
Total params: 41,600
Trainable params: 41,600
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
 - 1s - loss: 3.4188 - acc: 0.1638
Epoch 2/100
 - 0s - loss: 2.9098 - acc: 0.1803
Epoch 3/100
 - 0s - loss: 2.8836 - acc: 0.1813
Epoch 4/100
 - 0s - loss: 2.8528 - acc: 0.2041
Epoch 5/100
 - 0s - loss: 2.8276 - acc: 0.2089
Epoch 6/100
 - 0s - loss: 2.7925 - acc: 0.2190
Epoch 7/100
 - 0s - loss: 2.7452 - acc: 0.2450
Epoch 8/100
 - 0s - loss: 2.6819 - acc: 0.2561
Epoch 9/100
 - 0s - loss: 2.6179 - acc: 0.2731
Epoch 10/100
 - 0s - loss: 2.5529 - acc: 0.2853
Epoch 11/100
 - 0s - loss: 2.4946 - acc: 0.2847
Epoch 12/100
 -

In [13]:
import numpy as np
from pickle import load
#from numpy import array
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        #encoded = encoded.reshape(1, encoded.shape[0], encoded.shape[1])
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        #print(yhat)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break 
        # append to input
        in_text += out_char
    return in_text
# load the model
model = load_model('/media/raul/datos/clases-pln/modelo-caracter/model.h5')
# load the mapping
mapping = load(open('/media/raul/datos/clases-pln/modelo-caracter/mapping.pkl', 'rb'))
print(mapping)
# test start of rhyme
print(generate_seq(model, mapping, 10, 'en ti', 25))
# test mid-line
print(generate_seq(model, mapping, 10, 'A tu', 25))
# test not in original
#print(generate_seq(model, mapping, 20, 'The queen', 25))

{'\n': 0, ' ': 1, ',': 2, '-': 3, '.': 4, ':': 5, ';': 6, 'A': 7, 'C': 8, 'D': 9, 'E': 10, 'L': 11, 'M': 12, 'O': 13, 'P': 14, 'Q': 15, 'R': 16, 'S': 17, 'T': 18, 'V': 19, 'a': 20, 'b': 21, 'c': 22, 'd': 23, 'e': 24, 'f': 25, 'g': 26, 'h': 27, 'i': 28, 'j': 29, 'l': 30, 'm': 31, 'n': 32, 'o': 33, 'p': 34, 'q': 35, 'r': 36, 's': 37, 't': 38, 'u': 39, 'v': 40, 'x': 41, 'y': 42, 'z': 43, 'á': 44, 'é': 45, 'í': 46, 'ñ': 47, 'ó': 48, 'ú': 49}
en tirasas de as pileta sus mo
A turde con, rusos, prenas, b
